# BASICS 03 - One layer of HOTS in PyTorch

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import tonic, torch
from utils import timesurface, get_loader, get_dataset_info, plot_kernels, plot_weight_distribution
import matplotlib.pyplot as plt
from tqdm import tqdm
%cd ../hots
from layer import hotslayer

print(f' Tonic version installed -> {tonic.__version__}')

/home/antoine/homhots/hotsline/hots
 Tonic version installed -> 1.0.15


In [17]:
transform = tonic.transforms.NumpyAsType(int)
dataset = tonic.datasets.DVSGesture(save_to='../../Data/', train=True)#, transform=transform)
loader = get_loader(dataset, kfold = None, shuffle=True)
#get_dataset_info(dataset, properties = ['time', 'mean_isi', 'nb_events']);
print(f'number of samples in the dataset: {len(loader)}')

number of samples in the dataset: 1077


In [18]:
transform = tonic.transforms.NumpyAsType(int)
slicing_time_window = 1000000 # microseconds
slicer = tonic.slicers.SliceByTime(time_window=slicing_time_window)
sliced_dataset = tonic.SlicedDataset(dataset, transform=transform, slicer=slicer, metadata_path='./metadata/gesture')

Read metadata from disk.


In [15]:
loader = get_loader(sliced_dataset, kfold = None, shuffle=True)
print(f'number of samples in the dataset: {len(loader)}')

number of samples in the dataset: 6417


In [20]:
events, label = sliced_dataset[1]

In [23]:
events, label = next(iter(loader))

TypeError: can't convert np.ndarray of type numpy.void. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [22]:
print(events, label)

[[     68      57       1 1000042]
 [     64      20       0 1000049]
 [     91      98       0 1000065]
 ...
 [     71      62       0 1999919]
 [     68      92       0 1999998]
 [    101     113       0 1999998]] 7


In [19]:
R = 4
n_pola = 2
ts_size1 = (2*R+1)**2*n_pola
n_neurons = 4
tau = 5e3#7e2
ts_size2 = (2*(2*R)+1)**2*n_neurons
ts_size3 = (2*(4*R)+1)**2*n_neurons*2
verbose = True

p_index = dataset.ordering.index('p')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if verbose: print(f'device -> {device}')

torch.set_default_tensor_type("torch.DoubleTensor")

with torch.no_grad():
    output_pola = []
    layer1 = hotslayer(ts_size1, n_neurons, device=device)
    layer1.learning_flag = False
    layer1.to(device)
    layer2 = hotslayer(ts_size2, n_neurons*2, device=device)
    layer2.learning_flag = False
    layer2.to(device)
    layer3 = hotslayer(ts_size3, n_neurons*4, device=device)
    layer3.learning_flag = False
    layer3.to(device)
    for events, target in tqdm(loader):
        all_ts = timesurface(events.squeeze(0), dataset.sensor_size, dataset.ordering, surface_dimensions=[2*R+1,2*R+1], device=device)
        print('done TS 1')
        print(all_ts.shape
        n_star = layer1(all_ts.to(device).squeeze(0))
        events[0,:,p_index] = n_star.cpu()
        sensor_size = [dataset.sensor_size[0], dataset.sensor_size[1], n_neurons]
        all_ts = timesurface(events.squeeze(0), sensor_size, dataset.ordering, surface_dimensions=[2*(2*R)+1,2*(2*R)+1], device=device)
        print('done TS 2')
        n_star = layer2(all_ts.to(device).squeeze(0))
        sensor_size = [dataset.sensor_size[0], dataset.sensor_size[1], n_neurons*2]
        events[0,:,p_index] = n_star.cpu()
        all_ts = timesurface(events.squeeze(0), sensor_size, dataset.ordering, surface_dimensions=[2*(4*R)+1,2*(4*R)+1], device=device)
        print('done TS 3')
        n_star = layer3(all_ts.to(device).squeeze(0))
        output_pola.append(n_star.cpu())

device -> cuda


  0%|                                                                                                                                                                                                                                                | 0/1077 [00:00<?, ?it/s]


TypeError: can't convert np.ndarray of type numpy.void. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [ ]:
plot_kernels(layer, n_pola, R, width = 30)

In [ ]:
plot_weight_distribution(layer)

In [22]:
print(output_pola[0].cpu())

tensor([ 5,  7,  5,  ...,  9, 17, 13])


In [23]:
print(events.shape)

torch.Size([1, 5299, 4])


In [7]:
print(ts_size1, ts_size2)

162 4624
